In [2]:
import pandas as pd

data=pd.read_csv('lyric_nouns.tsv', sep='\t', encoding = 'utf-8')
data.head()

,년도,가수,제목,장르,가사
0,19920000,잼,난 멈추지 않는다,댄스,"['이제', '다시', '시작', '내겐', '시간', '때론', '상처', '좌절..."
1,19920000,잼,우리 모두 사랑하자,댄스,"['모두', '사랑', '위', '모두', '춤', '인', '스텝', '속', '..."
2,19920000,잼,이유,발라드,"['어둠', '이제', '시간', '가슴속', '긴머리', '향', '향기', '잠..."
3,19920000,잼,18번가의 비밀,댄스,"['어둠', '속', '느낌', '거리', '비', '해지', '바람', '어깨',..."
4,19920000,잼,요즘 친구들,댄스,"['요즘', '친구', '흥', '정말', '친구', '말로', '자기', '다만'..."


In [7]:
lyric = data['가사']
print(len(lyric))
lyric.head()

5773


0    ['이제', '다시', '시작', '내겐', '시간', '때론', '상처', '좌절...
1    ['모두', '사랑', '위', '모두', '춤', '인', '스텝', '속', '...
2    ['어둠', '이제', '시간', '가슴속', '긴머리', '향', '향기', '잠...
3    ['어둠', '속', '느낌', '거리', '비', '해지', '바람', '어깨',...
4    ['요즘', '친구', '흥', '정말', '친구', '말로', '자기', '다만'...
Name: 가사, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

In [9]:
#TF-IDF vectorizer
tfv = TfidfVectorizer()
#transform
vec_text = tfv.fit_transform(lyric)
print(vec_text)

  (0, 6849)	0.3425151389188998
  (0, 6613)	0.18436352018582902
  (0, 6009)	0.44290019879366677
  (0, 8885)	0.3042868799725523
  (0, 11415)	0.0947004109715076
  (0, 1432)	0.11729757800911624
  (0, 8697)	0.18272779241114584
  (0, 5086)	0.09571718660144826
  (0, 2477)	0.12239621111522361
  (0, 5877)	0.17723540535109103
  (0, 1472)	0.28340004319860956
  (0, 5946)	0.42586361327845274
  (0, 1806)	0.31408219234462065
  (0, 7861)	0.296560311729245
  (1, 1206)	0.13052906018326857
  (1, 9589)	0.06308760850268687
  (1, 1936)	0.10317031558347337
  (1, 9330)	0.13190661032987788
  (1, 7785)	0.0949355890399528
  (1, 8100)	0.13874203316026842
  (1, 3635)	0.1367835475942323
  (1, 2025)	0.14093143375134082
  (1, 7065)	0.04558847338613197
  (1, 878)	0.090748610501516
  (1, 8129)	0.10924285941653497
  :	:
  (5771, 4872)	0.19886141766892115
  (5771, 3315)	0.05919046119371655
  (5771, 5086)	0.0862419558575344
  (5772, 1213)	0.46257148421046845
  (5772, 6137)	0.14721914632116365
  (5772, 3067)	0.111466349823

In [11]:
print(len(tfv.idf_))
tfv.idf_

11548


array([7.71521021, 6.32891585, 8.96797318, ..., 6.66538809, 8.96797318,
       8.96797318])

In [26]:
#setup kmeans clustering
kmeans = KMeans(n_clusters = 8, init='random', random_state=2)
#fit the data 
kmeans = kmeans.fit(vec_text)

In [27]:
labels=kmeans.labels_
print(labels)
print(len(labels))

[6 1 6 ... 5 0 0]
5773


In [33]:
clustered=pd.DataFrame(list(zip(data['가수'],data['제목'],labels)),columns=['singer','title','cluster'])

In [34]:
clustered

,singer,title,cluster
0,잼,난 멈추지 않는다,6
1,잼,우리 모두 사랑하자,1
2,잼,이유,6
3,잼,18번가의 비밀,7
4,잼,요즘 친구들,3
...,...,...,...
5768,업텐션 (UP10TION),니가 보여 (Waiting For You),5
5769,업텐션 (UP10TION),빈집 (텅) (Empty House),0
5770,엔플라잉 (N.Flying),From You,5
5771,BLACKPINK,Lovesick Girls,0


In [35]:
clustered.to_csv('clustered.tsv', index=False, sep='\t')

In [30]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : 사랑, 사람, 모두, 지금, 우린, 남자, 정말, 어디, 노래, 다른
1 : 사랑, 사람, 눈물, 가슴, 이별, 마음, 다시, 하나, 이제, 혼자
2 : 나나, 사랑, 라라라라, 사람, 자꾸, 난나, 나빌레, 마음, 오늘, 동경
3 : 오늘, 마음, 생각, 하루, 매일, 사랑, 보고, 자꾸, 지금, 시간
4 : 순간, 느낌, 지금, 모든, 시간, 눈빛, 사랑, 마음, 기분, 점점
5 : 다시, 기억, 눈물, 사랑, 추억, 시간, 모습, 그때, 가슴, 이별
6 : 이제, 사랑, 모두, 시간, 이상, 다시, 조금, 지금, 모든, 마음
7 : 세상, 하나, 사랑, 모두, 모든, 위해, 항상, 사람, 하늘, 모습


In [42]:
clustered.groupby('cluster').count()

,singer,title
cluster,,
0,2204,2204
1,844,844
2,60,60
3,779,779
4,441,441
5,595,595
6,347,347
7,503,503


In [51]:
print(kmeans.inertia_)

5204.283489398575


In [31]:
kmeans.cluster_centers_

array([[5.26816122e-04, 9.87935774e-04, 0.00000000e+00, ...,
        8.83883146e-04, 8.02273344e-05, 2.67424448e-05],
       [0.00000000e+00, 7.83492434e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 3.43197489e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.97548452e-04, 4.51140614e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.58414176e-04, 4.07720682e-04, ...,
        8.68905767e-04, 0.00000000e+00, 0.00000000e+00]])

In [52]:
for i in range (2,11):    
    #setup kmeans clustering
    kmeans = KMeans(n_clusters = i, init='random', random_state=2)
    #fit the data 
    kmeans = kmeans.fit(vec_text)
    
    print(kmeans.inertia_)

5320.532834172951
5293.061493491409
5274.06997535524
5258.936588272998
5230.478867421905
5215.450186507204
5204.283489398575
5190.664781256765
5175.840285342683
